# EDA

## 1. Load cleaned data

In [1]:
# Standard imports
from pathlib import Path
import ast
import os
import sys

def set_project_root():
    notebooks_dir = Path.cwd()

    # Calculate the root directory of the project (go up three levels)
    project_root = notebooks_dir.parent.parent.parent

    if str(project_root) not in sys.path:
        print(f"The root directory of the project is: {project_root}")
        sys.path.append(str(project_root))

    return project_root

project_root = set_project_root()

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Third party imports
import numpy as np
import pandas as pd

# Local imports
from pipeline.src._csv_utils import DataPathCleaningManager

The root directory of the project is: d:\UserData karol\Documents\Programming\Data Science\Data Engineering\Rent comparisions\Home Market Harvester


In [2]:
data_timeplace = os.getenv("MARKET_OFFERS_TIMEPLACE")
if data_timeplace is None:
    raise ValueError("The environment variable MARKET_OFFERS_TIMEPLACE is not set.")

data_path_manager = DataPathCleaningManager(data_timeplace, project_root)

try:
    df_olx = data_path_manager.load_df(domain="olx", is_cleaned=True)
except FileNotFoundError as e:
    print(e)
    df_olx = None

try:
    df_otodom = data_path_manager.load_df(domain="otodom", is_cleaned=True)
except FileNotFoundError as e:
    print(e)
    df_otodom = None

[Errno 2] No such file or directory: 'd:\\UserData karol\\Documents\\Programming\\Data Science\\Data Engineering\\Rent comparisions\\Home Market Harvester\\data\\cleaned\\2024_02_08_13_11_12_Mierzęcice__Będziński__Śląskie\\otodom.pl.csv'


In [3]:
if df_olx is None and df_otodom is None:
    raise ValueError("No dataframes were loaded.")

### 1.1 OLX

In [4]:
df_olx

,link,title,price,summary_description,ownership,floor_level,is_furnished,building_type,square_meters,number_of_rooms,rent,voivodeship,city,street
0,https://www.olx.pl/d/oferta/wynajme-nowy-apart...,Wynajmę nowy apartament w domu czterorodzinnym,2500.0,"OpisWynajmę nowy apartament o powierzchni 60,3...",private,1,True,NaN,60,3,500.0,Śląskie,Przeczyce,<NA>
1,https://www.olx.pl/d/oferta/wynajme-mieszkanie...,Wynajmę mieszkanie w Mierzęcicach,2600.0,"OpisWynajmę mieszkanie 2 pokojowe, na 1 piętrz...",private,1,True,NaN,49,2,1.0,Śląskie,Mierzęcice,<NA>
2,https://www.olx.pl/d/oferta/mieszkanie-47m2-os...,Mieszkanie 47m2 Osiedle Mierzęcice,1700.0,"OpisWitam, wynajmę mieszkanie! Mieszkanie jest...",private,1,True,NaN,47,2,1.0,Śląskie,Nowa Wieś,<NA>


In [5]:
df_olx.dtypes

link                    string
title                   string
price                  float64
summary_description     string
ownership               object
floor_level              Int64
is_furnished              bool
building_type           object
square_meters            Int64
number_of_rooms          Int64
rent                   float64
voivodeship             string
city                    string
street                  string
dtype: object

In [6]:
if df_olx is not None:
    df_olx.columns

### 1.2 otodom

In [7]:
if df_otodom is not None:
    df_otodom.sample(5)

In [8]:
if df_otodom is not None:
    df_otodom.columns

### 1.3 Combined

In [9]:
def safely_convert_dtypes(df, dtype_specs) -> pd.DataFrame:
    """
    Converts column data types in a DataFrame according to the specified data types,
    handling exceptions gracefully.
    
    Args:
    - df: pandas DataFrame to convert.
    - dtype_specs: Dictionary specifying the target data type for each column.
    """
    # Transform keys in dtype_specs if necessary
    transformed_dtype_specs = {
        ast.literal_eval(key) if isinstance(key, str) and key.startswith('(') else key: dtype
        for key, dtype in dtype_specs.items()
    }

    for column, target_dtype in transformed_dtype_specs.items():
        try:
            # Ensure column exists in DataFrame before attempting conversion
            if column in df.columns:
                df[column] = df[column].astype(target_dtype)
            else:
                print(f"Warning: Column {column} does not exist in the DataFrame.")
        except ValueError as e:
            print(f"Warning: Could not convert column {column} to {target_dtype}. Error: {e}")
    
    return df

def ensure_multiindex(df, combined_schema) -> pd.DataFrame:
    """
    Ensures the DataFrame's columns are in MultiIndex format according to the combined schema.
    
    Args:
    - df: DataFrame to adjust.
    - combined_schema: The schema with which to align the DataFrame's columns.
    """
    # Convert column names to MultiIndex if they are not already
    if not isinstance(df.columns, pd.MultiIndex):
        multiindex_columns = [tuple(col.split(", ")) if ", " in col else (col, '') for col in combined_schema]
        df.columns = pd.MultiIndex.from_tuples(multiindex_columns)
    return df

def align_columns_to_schema(df, combined_schema) -> pd.DataFrame:
    """
    Aligns DataFrame columns to the combined schema, preserving existing columns and adding missing ones.
    
    Args:
    - df: DataFrame to align.
    - combined_schema: Schema to align the DataFrame's columns to.
    """
    # Generate the target column order from the schema
    target_columns = list(combined_schema.keys())
    
    # Identify missing columns and fill them appropriately
    missing_columns = [col for col in target_columns if col not in df.columns]
    for col in missing_columns:
        df[col] = np.nan  # or False for boolean columns, as appropriate
    
    # Reorder the DataFrame to match the target column order, including only the columns present in the schema
    df = df.reindex(columns=[col for col in target_columns if col in df.columns or col in missing_columns])
    
    return df

def transform_olx(df_olx: pd.DataFrame, combined_df_schema_json: dict) -> pd.DataFrame:
    """
    Transforms the df_olx DataFrame to align with the combined DataFrame schema,
    including converting column names to a MultiIndex format, filling missing columns,
    adding calculated columns, and ensuring data types match the combined schema.

    Args:
    - df_olx (pd.DataFrame): DataFrame containing data from OLX.
    - combined_df_schema_json (dict): Schema definition for the combined DataFrame,
                                      including data types and column structure.

    Returns:
    - pd.DataFrame: Transformed df_olx aligned with the combined DataFrame schema.
    """

    # Step 1: Create a mapping
    column_mapping = {
        'link': ('listing', 'link'),
        'title': ('listing', 'title'),
        'price': ('pricing', 'price'),
        'rent': ('pricing', 'rent'),
        'summary_description': ('listing', 'summary_description'),
        'ownership': ('legal_and_availability', 'ownership'),
        'floor_level': ('size', 'floor'),
        'is_furnished': ('equipment', 'furniture'),
        'building_type': ('type_and_year', 'building_type'),
        'square_meters': ('size', 'square_meters'),
        'number_of_rooms': ('size', 'number_of_rooms'),
        'voivodeship': ('location', 'voivodeship'),
        'city': ('location', 'city'),
        'street': ('location', 'street')
    }


    # Step 2: Modify df_olx to have a MultiIndex
    df_olx.columns = pd.MultiIndex.from_tuples([column_mapping.get(col, (col, '')) for col in df_olx.columns])

    # Step 3: Identify and fill missing columns 
    # in df_olx based on the combined schema
    combined_df_columns_names = [ast.literal_eval(key) if isinstance(key, str) and key.startswith('(') else key for key in combined_df_schema_json["dtypes"].keys()]
    missing_columns = set(combined_df_columns_names) - set(df_olx.columns)

    for col in missing_columns:
        if col in [('equipment', 'furniture'),]:  # Add other boolean columns if any
            df_olx[col] = False
        else:
            df_olx[col] = np.nan

    # Step 5: Reorder df_olx columns to match the combined DataFrame schema
    df_olx = df_olx.reindex(columns=combined_df_columns_names)

    # Step 6: Add calculated columns
    df_olx[('pricing', 'total_rent')] = df_olx[('pricing', 'price')].add(df_olx[('pricing', 'rent')], fill_value=0)

    df_olx[('location', 'complete_address')] = df_olx.apply(
        lambda row: ', '.join(
            value for value in [row[('location', 'street')], row[('location', 'city')], row[('location', 'voivodeship')]]
            if not pd.isna(value)
        ),
        axis=1
    )

    # Step 7: Fill NaNs for specified columns and replace NaNs with appropriate values
    columns_to_fill_false = [
        ('size', 'attic'),
        ('amenities', 'elevator'),
        ('amenities', 'parking_space'),
        ('equipment', 'no_information'),
        ('equipment', 'stove'),
        ('equipment', 'fridge'),
        ('equipment', 'oven'),
        ('equipment', 'washing_machine'),
        ('equipment', 'TV'),
        ('equipment', 'dishwasher'),
        ('media_types', 'internet'),
        ('media_types', 'telephone'),
        ('media_types', 'cable_TV'),
        ('heating', 'electric'),
        ('heating', 'gas'),
        ('heating', 'other'),
        ('heating', 'boiler_room'),
        ('heating', 'district'),
        ('heating', 'tile_stove'),
        ('security', 'intercom_or_video_intercom'),
        ('security', 'anti_burglary_doors_or_windows'),
        ('security', 'monitoring_or_security'),
        ('security', 'anti_burglary_roller_blinds'),
        ('security', 'alarm_system'),
        ('security', 'enclosed_area'),
        ('windows', 'aluminum'),
        ('windows', 'wooden'),
        ('windows', 'plastic'),
        ('building_material', 'concrete'),
        ('building_material', 'aerated_concrete'),
        ('building_material', 'brick'),
        ('building_material', 'wood'),
        ('building_material', 'other'),
        ('building_material', 'lightweight_aggregate'),
        ('building_material', 'hollow_brick'),
        ('building_material', 'silicate'),
        ('building_material', 'large_panel'),
        ('building_material', 'reinforced_concrete'),
        ('additional_information', 'duplex'),
        ('additional_information', 'air_conditioning'),
        ('additional_information', 'separate_kitchen'),
        ('additional_information', 'basement'),
        ('additional_information', 'utility_room'),
        ('additional_information', 'non_smokers_only'),
    ] 
    for col in columns_to_fill_false:
        df_olx[col] = df_olx[col].fillna(False)

    columns_to_fill_true = [
        ('media_types', 'no_information'),
        ('heating', 'no_information'),
        ('security', 'no_information'),
        ('windows', 'no_information'),
        ('building_material', 'no_information'),
        ('additional_information', 'no_information'),
    ]

    for col in columns_to_fill_true:
        df_olx[col] = df_olx[col].fillna(True)

    # Step 8: Safely convert data types according to the combined schema
    df_olx = safely_convert_dtypes(df_olx, combined_df_schema_json["dtypes"])

    return df_olx

def transform_otodom(df_otodom: pd.DataFrame, combined_df_schema_json: dict) -> pd.DataFrame:
    """
    Transforms the df_otodom DataFrame to align with the combined DataFrame schema,
    including filling missing columns and ensuring data types match the combined schema.

    Args:
    - df_otodom (pd.DataFrame): DataFrame containing data from Otodom.
    - combined_df_schema_json (dict): Schema definition for the combined DataFrame,
                                      including data types and column structure.

    Returns:
    - pd.DataFrame: Transformed df_otodom aligned with the combined DataFrame schema.
    """

    # Step 1: Identify and fill missing columns in df_otodom based on the combined schema
    combined_df_columns_names = [
        ast.literal_eval(key) if isinstance(key, str) and key.startswith('(') else key for key in combined_df_schema_json["dtypes"].keys()
        ]

    # Step 2: Reorder otodom_pl columns to match the combined DataFrame schema
    df_otodom = df_otodom.reindex(columns=combined_df_columns_names)

    # Step 3: Safely convert data types according to the combined schema
    df_otodom = safely_convert_dtypes(df_otodom, combined_df_schema_json["dtypes"])

    return df_otodom

def transform_combined_df(combined_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds calculated columns to the combined DataFrame and reorders the columns.
    
    Args:
    - combined_df: DataFrame to transform.
    
    Returns:
    - pd.DataFrame: The transformed combined DataFrame.
    """

    # Step 1: Add deposit_ratio column
    combined_df.loc[:, ("pricing", "deposit_ratio")] = np.where(
        combined_df["pricing"]["total_rent"] != 0,
        (combined_df["pricing"]["deposit"] / combined_df["pricing"]["total_rent"]).round(2),
        np.nan  # or 0, depending on your preference for default value
    )

    # Step 2: Add total price per square meter column
    combined_df.loc[:, ("pricing", "total_rent_sqm")] = (combined_df['pricing']['total_rent'] / combined_df['size']['square_meters']).replace([np.inf, -np.inf], np.nan)

    # Step 3: Reorder columns
    columns = combined_df.columns.tolist()

    combined_df = combined_df[columns]

    return combined_df

def combine_olx_otodom(df_olx: pd.DataFrame, df_otodom: pd.DataFrame) -> pd.DataFrame:
    """
    Combines the OLX and Otodom DataFrames into a single DataFrame.
    """

    combined_df = pd.concat([df_otodom, df_olx], ignore_index=True)

    return combined_df

def create_combined_df(df_olx: pd.DataFrame, df_otodom: pd.DataFrame, combined_df_schema_json: dict) -> pd.DataFrame:
    """
    Creates the combined DataFrame by transforming the OLX and Otodom DataFrames and combining them.
    
    Args:
    - df_olx: DataFrame containing OLX data.
    - df_otodom: DataFrame containing Otodom data.
    
    Returns:
    - pd.DataFrame: The combined DataFrame.
    
    Raises:
    - ValueError: If both input DataFrames are None.
    """

    if df_olx is None and df_otodom is None:
        raise ValueError("Both dataframes are None.")

    if df_olx is not None:
        df_olx = transform_olx(df_olx, combined_df_schema_json)
    if df_otodom is not None:
        df_otodom = transform_otodom(df_otodom, combined_df_schema_json)

    combined_df = None
    if df_olx is not None and df_otodom is not None:
        combined_df = combine_olx_otodom(df_olx, df_otodom)
    elif df_olx is not None:
        combined_df = df_olx
    else:
        combined_df = df_otodom

    combined_df = transform_combined_df(combined_df)

    return combined_df

combined_df_schema_json = data_path_manager.load_schema("combined")

combined_df = create_combined_df(df_olx, df_otodom, combined_df_schema_json)

combined_df.tail()

C:\Users\karol\AppData\Local\Temp\ipykernel_9408\66517948.py:238: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined_df.loc[:, ("pricing", "total_rent_sqm")] = (combined_df['pricing']['total_rent'] / combined_df['size']['square_meters']).replace([np.inf, -np.inf], np.nan)


listing  \
                                                link   
0  https://www.olx.pl/d/oferta/wynajme-nowy-apart...   
1  https://www.olx.pl/d/oferta/wynajme-mieszkanie...   
2  https://www.olx.pl/d/oferta/mieszkanie-47m2-os...   

                                                   \
                                            title   
0  Wynajmę nowy apartament w domu czterorodzinnym   
1               Wynajmę mieszkanie w Mierzęcicach   
2              Mieszkanie 47m2 Osiedle Mierzęcice   

                                                                    pricing  \
                                 summary_description remote_service   price   
0  OpisWynajmę nowy apartament o powierzchni 60,3...           <NA>  2500.0   
1  OpisWynajmę mieszkanie 2 pokojowe, na 1 piętrz...           <NA>  2600.0   
2  OpisWitam, wynajmę mieszkanie! Mieszkanie jest...           <NA>  1700.0   

                                                          ...  \
    rent total_rent total_rent_sqm deposit deposit_ratio  ...   
0  500.0     3000.0           50.0     NaN           NaN  ...   
1    1.0     2601.0      53.081633     NaN           NaN  ...   
2    1.0     1701.0      36.191489     NaN           NaN  ...   

  building_material                                 additional_information  \
           silicate large_panel reinforced_concrete         no_information   
0             False       False               False                   True   
1             False       False               False                   True   
2             False       False               False                   True   

                                                                  \
  duplex air_conditioning separate_kitchen basement utility_room   
0  False            False            False    False        False   
1  False            False            False    False        False   
2  False            False            False    False        False   

                    
  non_smokers_only  
0            False  
1            False  
2            False  

[3 rows x 75 columns]

In [10]:
pd.reset_option('display.max_rows')

In [11]:
combined_df.dtypes.to_dict()

{('listing', 'link'): string[python],
 ('listing', 'title'): string[python],
 ('listing', 'summary_description'): string[python],
 ('listing', 'remote_service'): string[python],
 ('pricing', 'price'): dtype('float64'),
 ('pricing', 'rent'): dtype('float64'),
 ('pricing', 'total_rent'): dtype('float64'),
 ('pricing', 'total_rent_sqm'): Float64Dtype(),
 ('pricing', 'deposit'): dtype('float64'),
 ('pricing', 'deposit_ratio'): dtype('float64'),
 ('location', 'complete_address'): string[python],
 ('location', 'street'): string[python],
 ('location', 'city'): string[python],
 ('location', 'voivodeship'): string[python],
 ('size', 'square_meters'): Int64Dtype(),
 ('size', 'number_of_rooms'): Int64Dtype(),
 ('size', 'floor'): Int64Dtype(),
 ('size', 'attic'): dtype('bool'),
 ('size', 'building_floors'): Int64Dtype(),
 ('legal_and_availability', 'available_from'): string[python],
 ('legal_and_availability', 'completion'): string[python],
 ('legal_and_availability', 'ownership'): string[python],

Saving and checking combined df

In [12]:
data_path_manager.save_df(combined_df, domain="combined")

In [13]:
combined_df_loaded = data_path_manager.load_df(domain="combined", is_cleaned=True)

are_identical = combined_df_loaded.equals(combined_df)
if not are_identical:
    raise ValueError("The saved DataFrame is not identical to the original one.")
else:
    print("The saved DataFrame is identical to the original one.")

The saved DataFrame is identical to the original one.


In [14]:
if len(combined_df_loaded) < 5:
    print(f"The DataFrame has {len(combined_df_loaded)} rows.")
    combined_df_loaded.head()

The DataFrame has 3 rows.


In [15]:
if len(combined_df_loaded) >= 5:
    combined_df_loaded.sample(5).head()

In [16]:
combined_df_loaded[('listing', 'link')].duplicated().sum()

0

In [17]:
len(combined_df_loaded)

3